In [63]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
print("Device:", device)

Device: cuda


In [5]:
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    data_set = pickle.load(f)

In [6]:
data_set[0][1]

0

In [7]:
data = []
targets = []
for i in data_set:
  data.append(i[0])
  targets.append(i[1])

In [8]:
targets_array = np.array(targets)
targets_array_reshaped = targets_array.reshape(-1, 1)
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe.fit(targets_array_reshaped)

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [9]:
targets_ohe = ohe.transform(targets_array_reshaped)
print(targets_ohe)

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


In [10]:
print(data[-1])
print(targets[-1])
print(targets_ohe[-1])

[112. 112.   0.  64.   0. 112.  34. 144. 146. 112. 112. 179. 112. 112.
  73.   3.  12. 112. 146.  60.  45. 112.  73.  88. 117.   8. 159. 145.
 159. 112.  13.  80.  92. 159. 190. 112. 185. 190.   8.  73. 125.  77.
  12.   0. 117. 190. 185. 112. 125. 125.  45.  41.  92.  28.  92.  12.
 159.   0.  47.  12.  12.  88.  88.   0.  12. 112. 159.   8. 159.   0.
  12. 112. 125.  23.  47.  41. 159. 112.  45.  12.   8. 145.  47.  33.
  33.  33. 125.  73.   8.  13.  92.  88. 159. 120. 124. 185. 185.  77.
 125.  88.  13. 120. 120. 112. 112. 112. 159. 172.  12. 117.  47.  88.
  88.  36.  92.  33. 125.  88.  88.  88.  12. 112. 159.  88. 125. 119.
  47.  88.  92. 159. 185. 112.  77.  33. 125.  65. 125.  33. 125.  88.
  13. 120. 124. 112.  12.   0.   0.   0.  12. 112. 159. 145.   5.   0.
  88. 112.  12.  47.  88.   0.  12. 112. 159. 145.   5.   0.  12. 112.
 125. 125.  28. 159.  28.  38.  12. 117.  47. 114. 153.  88. 159.   8.
   8. 112.  77.  38.  12.   0. 117. 190. 185. 112.  77.  33.  41. 145.
 145. 

In [44]:
from collections import Counter

labels = [label for _, label in data_set]

class_counts = Counter(labels)
classes = []
print("Liczba klas:", len(class_counts))
for class_label, count in class_counts.items():
    classes.append(count)
classes = torch.tensor(classes)
print(classes)
class_weights = 1.0 / classes
class_weights = class_weights / class_weights.sum()
print(class_weights)

Liczba klas: 5
tensor([1630,  478,  154,  441,  236])
tensor([0.0391, 0.1332, 0.4135, 0.1444, 0.2698])


In [12]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [13]:
train_indices = np.random.rand(len(data_set))>0.3

In [14]:
train_indices

array([ True,  True,  True, ...,  True,  True, False])

In [78]:
def normalize(data):
    min_value = min(data)
    max_value = max(data)
    if min_value == max_value:
        # Jeśli wszystkie wartości są takie same, zwracamy listę jedynek
        return [1 for _ in data]
    else:
        normalized_data = [(x - min_value) / (max_value - min_value) for x in data]
        return normalized_data
    return normalized_data

In [86]:
data_normalized = []
for seq in data:
  data_normalized.append(normalize(seq))


In [87]:
data_train = []
targets_train = []
data_test = []
targets_test = []
for i in range(len(data_set)):
  if train_indices[i] == True:
    data_train.append(data_normalized[i])
    targets_train.append(targets_ohe[i])
  else:
    data_test.append(data_normalized[i])
    targets_test.append(targets_ohe[i])

In [88]:
print(data_train[100])

[0.4293193717277487, 0.4293193717277487, 0.4293193717277487, 0.005235602094240838, 0.9424083769633508, 0.015706806282722512, 0.8324607329842932, 0.02617801047120419, 0.7696335078534031, 0.031413612565445025, 0.06806282722513089, 0.10471204188481675, 0.24607329842931938, 0.041884816753926704, 0.6544502617801047, 0.9685863874345549, 0.4607329842931937, 0.3612565445026178, 0.9685863874345549, 0.6544502617801047, 0.4816753926701571, 0.6544502617801047, 0.6335078534031413, 0.31413612565445026, 0.06806282722513089, 0.4816753926701571, 0.18848167539267016, 0.24607329842931938, 0.02617801047120419, 0.8324607329842932, 0.41361256544502617, 0.41361256544502617, 0.041884816753926704, 0.6492146596858639, 0.6492146596858639, 0.9685863874345549, 0.5235602094240838, 0.010471204188481676, 0.41361256544502617, 0.38219895287958117, 0.3507853403141361, 0.6178010471204188, 0.06806282722513089, 0.041884816753926704, 0.9685863874345549, 0.5549738219895288, 0.41361256544502617, 0.41361256544502617, 0.6282722

In [16]:
# dataset_size = len(data_set)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size


# train_set = VariableLenDataset(data[:train_indices], targets[:train_indices])
# test_set = VariableLenDataset(data[train_indices:], targets[train_indices:])

In [89]:
train_set = VariableLenDataset(data_train, targets_train)
test_set = VariableLenDataset(data_test, targets_test)

In [90]:
print(train_set[2])

([0.3526315789473684, 0.531578947368421, 0.7842105263157895, 0.7842105263157895, 0.7736842105263158, 0.34210526315789475, 0.7736842105263158, 0.7842105263157895, 0.4368421052631579, 0.005263157894736842, 0.4368421052631579, 0.531578947368421, 0.18421052631578946, 0.7, 0.9526315789473684, 0.3473684210526316, 0.4263157894736842, 0.43157894736842106, 0.6947368421052632, 0.2789473684210526, 0.18421052631578946, 0.2789473684210526, 0.34210526315789475, 0.2789473684210526, 0.021052631578947368, 0.3526315789473684, 0.7789473684210526, 0.11052631578947368, 0.02631578947368421, 0.7, 0.7, 0.531578947368421, 0.5894736842105263, 0.39473684210526316, 0.5842105263157895, 0.32105263157894737, 0.48947368421052634, 0.3473684210526316, 0.531578947368421, 1.0, 0.23684210526315788, 0.04736842105263158, 0.031578947368421054, 0.4052631578947368, 0.16842105263157894, 0.8421052631578947, 0.031578947368421054, 0.6578947368421053, 0.02631578947368421, 0.06842105263157895, 0.2736842105263158, 0.8315789473684211,

In [19]:
pad = -100

# def pad_collate(batch, pad_value=pad):
#     xx, yy = zip(*batch)
#     xx = [torch.tensor(x) for x in xx]
#     x_lens = [len(x) for x in xx]
#     yy_pad = torch.tensor(yy)
#     y_lens = [1] * len(yy)
#     # print(xx)
#     # print(yy)

#     xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
#     # yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

#     return xx_pad, yy_pad, x_lens, y_lens

def pad_collate(batch, pad_value=pad):
    try:
        xx, yy = zip(*batch)
    except ValueError as e:
        print(f"Error while unpacking batch: {e}")
        print(f"Batch content: {batch}")
        raise

    xx = [torch.tensor(x) for x in xx]
    x_lens = [len(x) for x in xx]
    yy_pad = torch.tensor(yy)
    y_lens = [1] * len(yy)

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [91]:
batch_size=256

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [21]:
# dataset_size = len(data)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size

# train_set, test_set = random_split(data, [train_size, test_size])

# batch_size=4
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [22]:
data_set[0]


(array([ -1.,  -1.,  -1., ...,  78.,  40., 144.]), 0)

In [23]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

4756
5322


In [92]:
next(iter(train_loader))

(tensor([[   0.0000,    0.0000,    0.0000,  ..., -100.0000, -100.0000,
          -100.0000],
         [   0.0000,    0.0000,    0.0000,  ..., -100.0000, -100.0000,
          -100.0000],
         [   0.0000,    0.0000,    0.0000,  ..., -100.0000, -100.0000,
          -100.0000],
         ...,
         [   0.4294,    0.2294,    0.2294,  ..., -100.0000, -100.0000,
          -100.0000],
         [   0.0000,    0.0000,    0.0000,  ..., -100.0000, -100.0000,
          -100.0000],
         [   0.0000,    0.0000,    0.0000,  ..., -100.0000, -100.0000,
          -100.0000]], dtype=torch.float64),
 tensor([[1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         ...,
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1.]], dtype=torch.float64),
 [180,
  403,
  48,
  44,
  156,
  421,
  411,
  350,
  193,
  969,
  519,
  44,
  1272,
  108,
  28,
  87,
  1925,
  300,
  195,
  216,
  258,
  16,
  104,
  44,
  282,
  212,
  584

In [25]:
# class LSTMRegressor(nn.Module):

#     def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
#         super().__init__()
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
#         if bidirectional:
#             self.bidirectional = 2
#         else:
#             self.bidirectional = 1
#         self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
#         self.fc = nn.Linear(320, out_size)
#         # self.fc = nn.Linear(hidden_size*90*self.bidirectional, out_size)

#     def init_hidden(self, batch_size):
#         hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
#         state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
#         return hidden, state

#     def forward(self, x, hidden):
#         # x = torch.transpose(x,0,1)
#         x = x.float()
#         all_outputs, hidden = self.lstm(x, hidden)
#         # all_outputs = torch.transpose(all_outputs,0,1)
#         # out = torch.flatten(all_outputs,1)
#         # output, _ = pad_packed_sequence(all_outputs, batch_first=True)
#         # output = output[:, -1, :]

#         h_n, _ = hidden
#         h_n = torch.flatten(h_n, 1)  # Flatten the hidden state
#         # print(h_n[0])
#         x = self.fc(h_n)

#         # hidden = torch.flatten(hidden,1)
#         # print(hidden[0])
#         # x = self.fc(hidden)
#         return x, hidden

# model = LSTMRegressor(1,5,2,5).to(device)
# model

In [26]:
# class LSTMRegressor(nn.Module):

#     def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
#         super().__init__()
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
#         if bidirectional:
#             self.bidirectional = 2
#         else:
#             self.bidirectional = 1
#         self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
#         self.fc = nn.Linear(hidden_size * self.bidirectional, out_size)


#     def init_hidden(self, batch_size):
#         hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
#         state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
#         return hidden, state

#     def forward(self, x, hidden):
#         x = x.float()
#         all_outputs, hidden = self.lstm(x, hidden)
#         h_n, _ = hidden
#         # h_n = torch.flatten(h_n, 1)
#         # print(h_n)
#         x = self.fc(h_n)
#         return x, hidden

# model = LSTMRegressor(1,5,2,5).to(device)
# model

In [97]:
class LSTMClassifier(nn.Module): #Coś nie tak

    def __init__(self, input_size, hidden_size, num_layers, out_size, lin_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size * self.bidirectional, lin_size)
        self.fc2 = nn.Linear(lin_size,out_size)


    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        h_n, _ = hidden
        # h_n = torch.flatten(h_n, 1)
        # print(h_n)
        x = F.relu(self.fc(h_n[-1]))
        x = F.relu(self.fc2(x))
        # x = F.softmax(x, dim=1)
        # print(x)
        return x, hidden

model = LSTMClassifier(1,5,2,5,5).to(device)
model

LSTMClassifier(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=5, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=5, bias=True)
)

In [99]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)
loss_fun = nn.CrossEntropyLoss(weight=class_weights.to(device))

In [104]:
model = LSTMClassifier(1,64,5,5, 50).to(device)
num_epochs = 1000

for epoch in range(num_epochs):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)

        x = torch.transpose(x, 0, 1)
        # print(x)
        preds, _ = model(x, (hidden, state))
        # print(preds.size())
        # preds = torch.transpose(preds, 0, 1)


        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        optimizer.zero_grad()
        # mask = targets != pad
        # print(preds.size())
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")
        true_preds, num_preds = 0., 0.
        with torch.no_grad():
          for x, targets, x_len, target_len in test_loader:
              x = x.to(device).unsqueeze(2)
              targets = targets.to(device)
              hidden, state = model.init_hidden(x.shape[0])
              hidden, state = hidden.to(device), state.to(device)

              x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
              preds_packed, _ = model(x_packed, (hidden, state))
              preds, _ = model(x_packed, (hidden, state))
              # preds = torch.transpose(preds, 0, 1)

              # preds = preds.squeeze(2)
              # preds = preds.mean(dim=1)
              preds = torch.argmax(preds, dim=1)
              targets = torch.argmax(targets, dim=1)
              true_preds += (preds == targets).sum().item()
              num_preds += targets.shape[0]
        acc = true_preds / num_preds
        print(f"Accuracy: {100.0*acc:4.2f}%")


Epoch: 0, loss: 0.173
Accuracy: 54.78%
Epoch: 10, loss: 0.191
Accuracy: 54.78%
Epoch: 20, loss: 0.153
Accuracy: 54.78%
Epoch: 30, loss: 0.16
Accuracy: 54.78%
Epoch: 40, loss: 0.231
Accuracy: 54.78%
Epoch: 50, loss: 0.16
Accuracy: 54.78%
Epoch: 60, loss: 0.15
Accuracy: 54.78%
Epoch: 70, loss: 0.226
Accuracy: 54.78%
Epoch: 80, loss: 0.199
Accuracy: 54.78%
Epoch: 90, loss: 0.17
Accuracy: 54.78%
Epoch: 100, loss: 0.18
Accuracy: 54.78%
Epoch: 110, loss: 0.134
Accuracy: 54.78%
Epoch: 120, loss: 0.154
Accuracy: 54.78%
Epoch: 130, loss: 0.166
Accuracy: 54.78%
Epoch: 140, loss: 0.13
Accuracy: 54.78%
Epoch: 150, loss: 0.173
Accuracy: 54.78%
Epoch: 160, loss: 0.178
Accuracy: 54.78%
Epoch: 170, loss: 0.161
Accuracy: 54.78%
Epoch: 180, loss: 0.176
Accuracy: 54.78%
Epoch: 190, loss: 0.149
Accuracy: 54.78%
Epoch: 200, loss: 0.174
Accuracy: 54.78%
Epoch: 210, loss: 0.184
Accuracy: 54.78%
Epoch: 220, loss: 0.159
Accuracy: 54.78%
Epoch: 230, loss: 0.15
Accuracy: 54.78%
Epoch: 240, loss: 0.126
Accuracy: 

In [102]:
model = LSTMClassifier(1,64,5,5, 50).to(device)
num_epochs = 1000
losses = []

for epoch in range(num_epochs):
    for x, targets, x_len, target_len in train_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        preds, _ = model(x_packed, (hidden, state))
        # preds = torch.transpose(preds, 0, 1)

        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")
        true_preds, num_preds = 0., 0.
        with torch.no_grad():
          for x, targets, x_len, target_len in test_loader:
              x = x.to(device).unsqueeze(2)
              targets = targets.to(device)
              hidden, state = model.init_hidden(x.shape[0])
              hidden, state = hidden.to(device), state.to(device)

              x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
              preds_packed, _ = model(x_packed, (hidden, state))
              preds, _ = model(x_packed, (hidden, state))
              # preds = torch.transpose(preds, 0, 1)

              # preds = preds.squeeze(2)
              # preds = preds.mean(dim=1)
              preds = torch.argmax(preds, dim=1)
              targets = torch.argmax(targets, dim=1)
              true_preds += (preds == targets).sum().item()
              num_preds += targets.shape[0]
        acc = true_preds / num_preds
        print(f"Accuracy: {100.0*acc:4.2f}%")


Epoch: 0, loss: 0.196
Accuracy: 13.40%
Epoch: 10, loss: 0.179
Accuracy: 13.40%
Epoch: 20, loss: 0.175
Accuracy: 13.40%
Epoch: 30, loss: 0.184
Accuracy: 13.40%
Epoch: 40, loss: 0.186
Accuracy: 13.40%
Epoch: 50, loss: 0.151
Accuracy: 13.40%
Epoch: 60, loss: 0.175
Accuracy: 13.40%
Epoch: 70, loss: 0.196
Accuracy: 13.40%
Epoch: 80, loss: 0.167
Accuracy: 13.40%
Epoch: 90, loss: 0.226
Accuracy: 13.40%
Epoch: 100, loss: 0.197
Accuracy: 13.40%
Epoch: 110, loss: 0.179
Accuracy: 13.40%
Epoch: 120, loss: 0.163
Accuracy: 13.40%
Epoch: 130, loss: 0.164
Accuracy: 13.40%
Epoch: 140, loss: 0.207
Accuracy: 13.40%
Epoch: 150, loss: 0.148
Accuracy: 13.40%
Epoch: 160, loss: 0.199
Accuracy: 13.40%
Epoch: 170, loss: 0.12
Accuracy: 13.40%
Epoch: 180, loss: 0.152
Accuracy: 13.40%
Epoch: 190, loss: 0.136
Accuracy: 13.40%
Epoch: 200, loss: 0.134
Accuracy: 13.40%
Epoch: 210, loss: 0.232
Accuracy: 13.40%
Epoch: 220, loss: 0.157
Accuracy: 13.40%
Epoch: 230, loss: 0.165
Accuracy: 13.40%
Epoch: 240, loss: 0.136
Accu

KeyboardInterrupt: 

In [101]:
true_preds, num_preds = 0., 0.

with torch.no_grad():
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[0])
        hidden, state = hidden.to(device), state.to(device)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        preds_packed, _ = model(x_packed, (hidden, state))
        preds, _ = model(x_packed, (hidden, state))
        # preds = torch.transpose(preds, 0, 1)

        # preds = preds.squeeze(2)
        # preds = preds.mean(dim=1)
        preds = torch.argmax(preds, dim=1)
        targets = torch.argmax(targets, dim=1)
        print("preds",preds)
        print(targets)
        true_preds += (preds == targets).sum().item()
        num_preds += targets.shape[0]
acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

preds tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,